In [11]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    #類別屬性，用在__bytes__

    #Private Property
    #轉成float可捕抓錯誤
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    #get
    @property
    def x(self):
        return self.__x

    #get
    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))
        #使得可以unpack
        #__repr__跟__str__也是用這個

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    #用xor來hash
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)
    
    #可選擇使用笛卡爾座標或極座標
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1] 
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    #不用建立instance就可以呼叫
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode) 
        return cls(*memv)


In [14]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [29]:
#x y只是被換了名稱，硬要(惡意)存取還是可
v1.__dict__, v1._Vector2d__x

({'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}, 3.0)

In [15]:
#__iter__
v2 = Vector2d(3, 4)
a, b = v2
a, b

(3.0, 4.0)

In [17]:
#__repr__
v3 = Vector2d(3, 4)
v3

Vector2d(3.0, 4.0)

In [18]:
#__repr__
v4 = Vector2d(3, 4)
v4_clone = eval(repr(v4))
v4 == v4_clone

True

In [19]:
#__str__
v5 = Vector2d(3, 4)
print(v5)

(3.0, 4.0)


In [22]:
#__abs__
v6 = Vector2d(3, 4)
abs(v6)

5.0

In [23]:

bool(Vector2d(3, 0)), bool(Vector2d(0, 0))

(True, False)

In [24]:
v7 = Vector2d(3,4)
v7_clone = Vector2d.frombytes(bytes(v7))
bytes(v7), v7 == v7_clone

(b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@', True)

In [25]:
#__angle__
Vector2d(0, 1).angle(),Vector2d(1, 0).angle()

(1.5707963267948966, 0.0)

In [26]:
#__format__
#顯示極座標
format(Vector2d(0, 1), 'p') 

'<1.0, 1.5707963267948966>'

In [27]:
#__hash__
v8 = Vector2d(3,4)
v9 = Vector2d(3.001, 4.001)

hash(v8), hash(v9)

(7, 3079)